# Projet de Maths
## Visualisation de données des tremblements de terre

In [1]:
# Importation des librairies
import pandas as pd
import geopandas
import folium
import geodatasets
import matplotlib.pyplot as plt

In [2]:
# Lecture du fichier csv

df1 = pd.read_csv("seismes_2014.csv")

df = df1.loc[:, ("instant", "lat", "lon", "pays", "mag", "profondeur")]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120108 entries, 0 to 120107
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   instant     120108 non-null  object 
 1   lat         120108 non-null  float64
 2   lon         120108 non-null  float64
 3   pays        119753 non-null  object 
 4   mag         120065 non-null  float64
 5   profondeur  120107 non-null  float64
dtypes: float64(4), object(2)
memory usage: 5.5+ MB


In [3]:
geometry = geopandas.points_from_xy(df.lon, df.lat)
geo_df = geopandas.GeoDataFrame(
    df[["instant", "lat", "lon", "pays", "mag", "profondeur"]], geometry=geometry
)

geo_df.head()

,instant,lat,lon,pays,mag,profondeur,geometry
0,2014-01-31 23:53:37.000,60.252000,-152.7081,Alaska,1.10,90.20,POINT (-152.70810 60.25200)
1,2014-01-31 23:48:35.452,37.070300,-115.1309,Nevada,1.33,0.00,POINT (-115.13090 37.07030)
2,2014-01-31 23:47:24.000,64.671700,-149.2528,Alaska,1.30,7.10,POINT (-149.25280 64.67170)
3,2014-01-31 23:30:54.000,63.188700,-148.9575,Alaska,0.80,96.50,POINT (-148.95750 63.18870)
4,2014-01-31 23:30:52.210,32.616833,-115.6925,Mexico,1.34,10.59,POINT (-115.69250 32.61683)


In [4]:
# On utilise le fond de carte OpenStreetMap
map = folium.Map(tiles="OpenStreetMap", zoom_start=9)

In [5]:
# Crée une liste de points des latitudes et longitudes

geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]

# Itère sur la liste des coordonnées et ajoute un point à la carte

i = 0
for coordinates in geo_df_list:
    # Assigne une couleur en fonction de la magnitude
    if geo_df.mag[i] < 0.2:
        type_color = "green"
    elif geo_df.mag[i] < 0.5:
        type_color = "blue"
    elif geo_df.mag[i] < 1.5:
        type_color = "orange"
    elif geo_df.mag[i] < 2.5:
        type_color = "pink"
    else:
        type_color = "purple"

    # Place le point sur la carte
    map.add_child(
        folium.Marker(
            location=coordinates,
            popup="instant: "
            + str(geo_df.instant[i])
            + "<br>"
            + "lat: "
            + str(geo_df.lat[i])
            + "<br>"
            + "lon: "
            + str(geo_df.lon[i])
            + "<br>"
            + "pays: "
            + str(geo_df.pays[i])
            + "<br>"
            + "mag: "
            + str(geo_df.mag[i])
            + "<br>"
            + "profondeur: "
            + str(geo_df.profondeur[i])
            + "<br>"
            + "Coordinates: "
            + str(geo_df_list[i]),
            icon=folium.Icon(color="%s" % type_color),
        )
    )
    i = i + 1

In [ ]:
map

In [ ]:
# Création d'une heat map avec les données fournis

from folium import plugins

map = folium.Map(location=[15, 30], tiles="Cartodb dark_matter", zoom_start=2)

heat_data = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]

heat_data
plugins.HeatMap(heat_data).add_to(map)

map